# **1. Data Extraction from MarketAUX API**

In [ ]:
# Basic Library
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

# BigQuery
from google.oauth2 import service_account
from pandas_gbq import to_gbq

# Logging
import logging
# Setup logger
logging.basicConfig(level=logging.INFO)

# Dotenv
from dotenv import load_dotenv

In [ ]:
# Base URL without page param
BASE_URL = "https://api.marketaux.com/v1/news/all"
API_TOKEN = "NEqVFjBNHIqWoYCYmABZTcDpfWhL5tIva3TmLLsc"

# Parameters assignment
published_after_date = (datetime.utcnow() - timedelta(days=90)).strftime("%Y-%m-%d")

# Parameters (token + language stays the same, page varies)
params = {
    "language": "en,id",
    "must_have_entities": "true",
    "published_after": published_after_date,
    "api_token": API_TOKEN
}

# Storage for all articles
all_articles = []

In [43]:
# Loop over 20 pages (pagination approach due to limit per day)
for page in range(1, 21):  # 1 to 20
    params["page"] = page
    response = requests.get(BASE_URL, params=params)
    
    print(f"Page {page} | Status code: {response.status_code}")
    logging.info(f"Fetched page {page} successfully.")

    if response.status_code != 200:
        print(f"Failed on page {page}")
        continue

    result = response.json()
    page_articles = result.get("data", [])

    all_articles.extend(page_articles)

    # Optional delay to avoid being flagged as abusive (depends on API rules)
    time.sleep(1)

INFO:root:Fetched page 1 successfully.


Page 1 | Status code: 200


INFO:root:Fetched page 2 successfully.


Page 2 | Status code: 200


INFO:root:Fetched page 3 successfully.


...

Page 18 | Status code: 200


INFO:root:Fetched page 19 successfully.


Page 19 | Status code: 200


INFO:root:Fetched page 20 successfully.


Page 20 | Status code: 200


# **2. Transform Data into Readily Available Use**

In [44]:
# Initialze the list holder for selected data
article_collection = []

# Select each columns for the new data holder
for article in all_articles:
    symbol = "unknown"
    name = "unknown"
    type = "undetermined"
    industry = "undetermined"

    if article.get("entities"):
        entity = article["entities"][0]
        symbol = entity.get("symbol", "unknown")
        name = entity.get("name", "unknown")
        type = entity.get("type", "undetermined")
        industry = entity.get("industry", "undetermined")

    article_collection.append({
        "title": article.get("title", ""),
        "keywords": article.get("keywords", ""),
        "url": article.get("url", ""),
        "image_url": article.get("image_url", ""),
        "published_at": article.get("published_at", ""),
        "source": article.get("source", ""),
        "language": article.get("language", ""),
        "symbol": symbol,
        "name": name,
        "type": type,
        "industry": industry
    })

In [45]:
# Changing the article_collection to a DataFrame (optional)
df = pd.DataFrame(article_collection)

# Enriching and deleting keywords column
df["list_of_keywords"] = df["keywords"].str.split(", ")
df.drop(columns=["keywords"], inplace=True)

# Changing data types for better suitability
df["title"] = df["title"].astype(str)
df["url"] = df["url"].astype(str)
df["image_url"] = df["image_url"].astype(str)
df["published_at"] = pd.to_datetime(df["published_at"]).dt.strftime("%Y-%m-%d")
df["source"] = df["source"].astype(str)
df["language"] = df["language"].astype(str)
df["symbol"] = df["symbol"].astype(str)
df["name"] = df["name"].astype(str)
df["type"] = df["type"].astype("category")
df["industry"] = df["industry"].astype("category")

In [46]:
# 2c. Save data into CSV
df.to_csv(f"../data/econNews-{datetime.now().strftime('%Y%m%d')}.csv", index=False)

# **3. Loading into BigQuery**

In [ ]:
def upload_to_bigquery(
    df: pd.DataFrame,
    table_id: str,
    project_id: str,
    credentials_path: str,
    if_exists: str = "append"):
    """
    Upload a pandas DataFrame to Google BigQuery using service account credentials.

    Parameters:
        df (pd.DataFrame): Cleaned and enriched data to upload.
        table_id (str): Full table path in BigQuery (e.g., dataset.table_name).
        project_id (str): Google Cloud project ID.
        credentials_path (str): Path to the service account JSON key file.
        if_exists (str): What to do if table exists. Options: 'fail', 'replace', 'append'.
                         Default is 'append'.

    Returns:
        None
    """
    try:
        logging.info(f"Authenticating using service account at: {credentials_path}")
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path
        )

        logging.info(f"Uploading to BigQuery table: {table_id} (mode: {if_exists})")
        to_gbq(
            dataframe=df,
            destination_table=table_id,
            project_id=project_id,
            if_exists=if_exists,
            credentials=credentials
        )
        logging.info("Upload completed successfully.")
    except Exception as e:
        logging.error(f"Failed to upload to BigQuery: {e}")

In [ ]:
# BigQuery configurations
PROJECT_ID = "econnews-daily-update "
TABLE_ID = "econnews_daily"
CREDENTIALS_PATH = "../config/econnews-daily-update-2e6ea15bf774.json"

upload_to_bigquery(
        df=df,
        table_id=TABLE_ID,
        project_id=PROJECT_ID,
        credentials_path=CREDENTIALS_PATH,
        if_exists="append")